In [1]:
import pandas as pd
import numpy as np

In [292]:
df_pp_bw_linked = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Source to Use Scenarios/Brackish to Power/df_pp_bw_linked.csv')

In [341]:
df_pp_bw_linked.columns

Index(['Unnamed: 0', 'PlantCode', 'Fuel', 'Cooling', 'Capacity', 'PPLatitude',
       'PPLongitude', 'WAvg', 'dist_km', 'site', 'state_alpha', 'county_nm',
       'Latitude', 'Longitude', 'well_depth_ft', 'well_yield', 'TDS_kgm3',
       'state', 'elec_price', 'lcow', 'recovery', 'capital', 'onm', 'elec_int',
       'tds_out', 'withm3s', 'metdemand'],
      dtype='object')

In [342]:
pp_group = df_pp_bw_linked.groupby('PlantCode')
site_group = df_pp_bw_linked.groupby('site')

One powerplant to many brackish water sites. 

In [470]:
empty = [None]*(len(list(pp_group.groups.keys())))
# Arbitrary cost $ / km /m3 ---> to be updated
transport_cost = 0.1

one_pp_many_bw  = pd.DataFrame( {"PlantCode": list(pp_group.groups.keys()),
                                "PPLatitude": empty,
                                "PPLongitude": empty,
                                "BWsites": empty,
                                "BWavailable": empty,
                                "PPDemandFracMet" : empty,
                                "LCOW":empty, 
                                "Distances_km": empty,
                                "TransportOnM":empty,
                                "NewLCOW": empty})

one_pp_many_bw.set_index("PlantCode",inplace=True)

# Some plant demands are 0 because the capacity is 0
check_demand = lambda x: 1 if x == 0 else x

for c_name in ['PPLatitude','PPLongitude']:
    one_pp_many_bw[c_name]=empty
    for key in pp_group.groups.keys():
        one_pp_many_bw.loc[key,c_name] = pp_group.get_group(key)[c_name].values[0]
        
for key in pp_group.groups.keys():
    sites = []
    indices = []
    sub_group = pp_group.get_group(key)
    for site in sub_group['site']:
        if site not in sites:
            index = sub_group[sub_group['site']==site].index[0]
            # print(index)
            sites.append(site)
            indices.append(index)
    
    one_pp_many_bw.loc[key,"BWsites"] =  {','.join(sub_group.loc[indices]['site'])}
    one_pp_many_bw.loc[key,"Distances_km"] = list(sub_group.loc[indices]['dist_km'])
    one_pp_many_bw.loc[key,"BWavailable"] = sum(sub_group.loc[indices]['recovery']*sub_group.loc[indices]['well_yield']/100)

    one_pp_many_bw.loc[key,"PPDemandFracMet"] = one_pp_many_bw.loc[key,"BWavailable"]/check_demand(sum(sub_group['withm3s']))
    one_pp_many_bw.loc[key,"LCOW"] = sum(sub_group['lcow'])
    
    bw_vol_annual = sub_group.loc[indices]['recovery']*sub_group.loc[indices]['well_yield']/100*365*24*60*60   # m3/year for each site
    one_pp_many_bw.loc[key,"TransportOnM"] = sum(transport_cost*sub_group.loc[indices]['dist_km']*bw_vol_annual)  # $ / year

    one_pp_many_bw.loc[key,"NewLCOW"] = one_pp_many_bw.loc[key,"LCOW"] + one_pp_many_bw.loc[key,"TransportOnM"]/sum(bw_vol_annual)
    
one_pp_many_bw
# print(bw_vol_annual)

,PPLatitude,PPLongitude,BWsites,BWavailable,PPDemandFracMet,LCOW,Distances_km,TransportOnM,NewLCOW
PlantCode,,,,,,,,,
3.0,31.0069,-88.0103,{FETH-304300N0880100.1},0.0158915,0.00027566,2.04048,[32.278286118002],1.61764e+06,5.2683
10.0,32.6017,-87.7811,{USGS-323012087492701},0.032539,0.00101305,0.851319,[11.64497526670747],1.19495e+06,2.01582
51.0,32.0306,-93.5692,"{USGS-314900093305001,USGS-315154093122001,USG...",0.181904,0.0699808,5.2345,"[24.32825136016584, 38.91478433810447, 12.5314...",1.53801e+07,7.91557
56.0,31.488,-87.9107,{USGS-315412087443401},0.0158821,0.0529577,1.05445,[48.707127667821716],2.43953e+06,5.92517
60.0,40.5809,-98.3124,"{FETH-405430N0983700.1,NURE-OR300243,NURE-OR30...",3.96293,0.855611,33.0025,"[44.53096757589436, 49.540355390768454, 42.156...",5.38637e+08,37.3124
...,...,...,...,...,...,...,...,...,...
60376.0,41.1489,-80.8516,"{USGS-410725080192901,USGS-412123080241801}",0.0406401,0.100148,1.94548,"[44.23084585457224, 43.90679282937148]",5.63914e+06,6.34546
60589.0,40.412,-78.855,"{USGS-402947078244701,USGS-403358079174201}",0.0945321,0.191912,1.72055,"[38.576596828195385, 40.950633482355045]",1.16274e+07,5.62085
60768.0,32.975,-112.694,"{AZ_GEOTH-AZ0425,USGS-324752112151201,USGS-325...",30.691,0.833012,179.966,"[26.671894308799327, 45.561411172467395, 22.39...",2.91965e+09,182.982


One brackish water site to many powerplants

In [477]:
empty = [None]*(len(list(site_group.groups.keys())))
one_bw_many_pp  = pd.DataFrame( {"BWsite": list(site_group.groups.keys()),
                                "BWLatitude": empty,
                                "BWLongitude": empty,
                                "PlantCode": empty,
                                "BWAvailable": empty,
                                "PPDemand" : empty,
                                "BWutilized": empty,
                                "LCOW":empty, 
                                "Distances_km": empty,
                                "TransportOnM":empty,
                                "NewLCOW": empty})

one_bw_many_pp.set_index("BWsite",inplace=True)
# Multiple listing of same plant code but different capacity

for key in site_group.groups.keys():
    pp_list = []
    indices = []
    sub_group = site_group.get_group(key)
    for pp in sub_group['PlantCode']:
        if pp not in pp_list:
            index = sub_group[sub_group['PlantCode']==pp].index[0]
            # print(index)
            pp_list.append(pp)
            indices.append(index)
    one_bw_many_pp.loc[key]["PlantCode"] =  pp_list
    one_bw_many_pp.loc[key]["BWAvailable"] = sub_group['recovery'].values[0]*sub_group['well_yield'].values[0]/100
    one_bw_many_pp.loc[key]["PPDemand"] = sum(sub_group['withm3s'])
    if one_bw_many_pp.loc[key]["BWAvailable"]- one_bw_many_pp.loc[key]["PPDemand"] <=0:
        one_bw_many_pp.loc[key]["BWutilized"] = 1
    else:
        one_bw_many_pp.loc[key]["BWutilized"] = (one_bw_many_pp.loc[key]["BWAvailable"]- one_bw_many_pp.loc[key]["PPDemand"])/one_bw_many_pp.loc[key]["BWAvailable"]
    one_bw_many_pp.loc[key]["LCOW"] = sum(sub_group['lcow'])
    one_bw_many_pp.loc[key,"Distances_km"] = list(sub_group.loc[indices]['dist_km'])
    one_bw_many_pp.loc[key]["TransportOnM"] = sum(transport_cost*sub_group.loc[indices]['dist_km']*one_bw_many_pp.loc[key]["BWAvailable"]*365*24*3600)  # $/yr

one_bw_many_pp["NewLCOW"] = one_bw_many_pp["LCOW"] + one_bw_many_pp["TransportOnM"]/(one_bw_many_pp.loc[key]["BWAvailable"]*365*24*3600)

for c_name in ['BWLatitude','BWLongitude']:
    one_bw_many_pp[c_name]=empty
for key in site_group.groups.keys():
    one_bw_many_pp.loc[key,"BWLatitude"] = sub_group["Latitude"].values[0]
    one_bw_many_pp.loc[key,"BWLongitude"] = sub_group["Longitude"].values[0]

one_bw_many_pp.head()

,BWLatitude,BWLongitude,PlantCode,BWAvailable,PPDemand,BWutilized,LCOW,Distances_km,TransportOnM,NewLCOW
BWsite,,,,,,,,,,
AZ_GEOTH-AZ0183,40.728,-112.201,[55129.0],0.0288949,0.163882,1,0.870494,[32.861529897587715],2.99444e+06,1.49746
AZ_GEOTH-AZ0274,40.728,-112.201,[55129.0],0.0617847,0.163882,1,0.762472,[20.012171921451962],3.89926e+06,1.57888
AZ_GEOTH-AZ0297,40.728,-112.201,[55129.0],0.110297,0.163882,1,0.911424,[21.53697251796861],7.49123e+06,2.4799
AZ_GEOTH-AZ0327,40.728,-112.201,"[8068.0, 55129.0]",0.0315991,0.405901,1,3.45139,"[48.36198153184309, 22.50876230597753]",7.06234e+06,4.93007
AZ_GEOTH-AZ0338,40.728,-112.201,"[8068.0, 55129.0]",0.0565085,0.405901,1,2.83842,"[47.17101542219742, 21.044020183004196]",1.21563e+07,5.38364


In [169]:
pp_group.get_group(108)['recovery']*pp_group.get_group(108)['well_yield']/100
test_BWavailable=pp_group.get_group(108)['recovery']*pp_group.get_group(108)['well_yield']/100
print(test_BWavailable)
test_BWavailable = sum(test_BWavailable)
print(test_BWavailable)

426    0.376293
428    0.210169
430    0.124894
dtype: float64
0.7113553397130297


In [44]:
test_metdemand_perc = pp_group.get_group(108)['withm3s']
print(test_metdemand_perc)
test_metdemand_perc = test_BWavailable / sum(pp_group.get_group(108)['withm3s'])
print(test_metdemand_perc )

426    0.19792
428    0.19792
430    0.19792
Name: withm3s, dtype: float64
1.1980532466189553


In [56]:
# Additional onm for transport

# Arbitrary cost $ / km /m3
transport_cost = 0.1
bw_vol_annual = pp_group.get_group(108)['well_yield']*(pp_group.get_group(108)['recovery'])/100*365*24*60*60 #m3/year
transport_onm_annual = transport_cost*pp_group.get_group(108)['dist_km']*bw_vol_annual # $/year

print(transport_onm_annual)

426    1.939086
428    2.805596
430    3.769638
dtype: float64


In [25]:
for key in pp_group.groups.keys():
    print(pp_group.get_group(key)['site'])


46    FETH-304300N0880100.1
47    FETH-304300N0880100.1
Name: site, dtype: object
1752    USGS-323012087492701
Name: site, dtype: object
1606    USGS-314900093305001
1615    USGS-315154093122001
1632    USGS-320259093420001
1656    USGS-321035093312001
1680    USGS-322052093374701
1707    USGS-322519093380201
Name: site, dtype: object
1616    USGS-315412087443401
Name: site, dtype: object
113     FETH-405430N0983700.1
114     FETH-405430N0983700.1
451             NURE-OR300243
452             NURE-OR300243
467             NURE-OR301761
468             NURE-OR301761
469             NURE-OR301762
470             NURE-OR301762
471             NURE-OR301771
472             NURE-OR301771
473             NURE-OR301776
474             NURE-OR301776
475             NURE-OR301777
476             NURE-OR301777
477             NURE-OR301781
478             NURE-OR301781
479             NURE-OR301782
480             NURE-OR301782
481             NURE-OR301783
482             NURE-OR301783
483     